### Modelo 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras import regularizers
#from keras.regularizers import l2

In [18]:
#Carga de Dataset desde bucket GCP
df = pd.read_csv('https://storage.googleapis.com/dataset_bucket_bot/glucosa.csv',delimiter=';',encoding='utf-8')
df['diabetic'] = df['diabetic'].replace('D', 1).replace('N', 0)

In [19]:
#Separacion del dataset en grupos de entrenamiento y grupos de testeo
X = df.drop('glucose', axis=1)
y = df['glucose']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [21]:
#Dataframes auxiliares para exportacion antes antes de transformaciones
X_train_df = X_train
X_test_df = X_test
y_train_df = y_train
y_test_df = y_test

In [29]:
#Exportacion de dataframes de entrenamiento y testeo
X_train_df.to_csv('X_train.csv',index=False)
X_test_df.to_csv('X_test.csv',index=False)
y_train_df.to_csv('y_train.csv',index=False)
y_test_df.to_csv('y_test.csv',index=False)

In [22]:
# Convertir los datos a tensores
X_train = tf.convert_to_tensor(X_train.values)
y_train = tf.convert_to_tensor(y_train.values)
#Estandarizando entradas
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


c:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [24]:
# Crear el modelo
model = keras.Sequential([
    layers.Dense(100, activation='relu', input_shape=[9]),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(1)
])
# Compilar el modelo
model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['mae', 'mse'])
# Entrenar el modelo
history = model.fit(X_train, y_train, validation_split=0.2, epochs=35, batch_size=8,verbose=2)
# Evaluar el modelo
loss, mae, mse = model.evaluate(X_test,y_test, verbose=2)
# Hacer predicciones con el modelo
predictions = model.predict(X_test)
#Realizando las prediccion
# 14 83 86	111	 91	96.262173	96	0	0	1
# 9	 66 73	104	105	96.390395	96	0	1	1
# 9	101 84	123	 83	96.003940	99	0	0	1
#x_input=[[14,86,111,91,96.26,96,0,0,1], #83
#         [9,73,104,105,96.39,96,0,0,1], #66
#         [9,84,123,83,96.00,99,0,0,1]] #101
x_input=[9,84,123,83,96.00,99,0,0,1]
xx = np.array(x_input).reshape(1,-1)
sc_input=sc.transform(xx)
pred = model.predict(sc_input)
pred[0][0] # mae 21.5786

Epoch 1/35
1358/1358 - 2s - loss: 1847.0325 - mae: 25.6699 - mse: 1847.0325 - val_loss: 1621.6149 - val_mae: 26.4434 - val_mse: 1621.6149 - 2s/epoch - 1ms/step
Epoch 2/35
1358/1358 - 1s - loss: 1516.6740 - mae: 22.7399 - mse: 1516.6740 - val_loss: 1614.7070 - val_mae: 26.6308 - val_mse: 1614.7070 - 1s/epoch - 850us/step
Epoch 3/35
1358/1358 - 1s - loss: 1503.1643 - mae: 22.4029 - mse: 1503.1643 - val_loss: 1519.7612 - val_mae: 21.7934 - val_mse: 1519.7612 - 1s/epoch - 850us/step
Epoch 4/35
1358/1358 - 1s - loss: 1499.7196 - mae: 22.3578 - mse: 1499.7196 - val_loss: 1510.5154 - val_mae: 22.3628 - val_mse: 1510.5154 - 1s/epoch - 836us/step
Epoch 5/35
1358/1358 - 1s - loss: 1503.8530 - mae: 22.1413 - mse: 1503.8530 - val_loss: 1512.9503 - val_mae: 21.9450 - val_mse: 1512.9503 - 1s/epoch - 843us/step
Epoch 6/35
1358/1358 - 1s - loss: 1496.2960 - mae: 22.1228 - mse: 1496.2960 - val_loss: 1562.3792 - val_mae: 24.7291 - val_mse: 1562.3792 - 1s/epoch - 842us/step
Epoch 7/35
1358/1358 - 1s - lo

101.0474

In [25]:
# Guardando el modelo
model.save('model4.h5')

In [26]:
# Cargando modelo grabado
new_model = keras.models.load_model('model4.h5')

In [30]:
# Realizando prediccion empleando el modelo guardado
x_input=[[14,86,111,91,96.26,96,0,0,1], #83
         [9,73,104,105,96.39,96,0,0,1], #66
         [9,84,123,83,96.00,99,0,0,1]] #101
sc_input=sc.transform(x_input)
pred1 = new_model.predict(sc_input)
pred1



1/1 [==============================] - 0s 16ms/step


array([[ 99.44219],
       [ 65.37765],
       [101.04738]], dtype=float32)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
df_historia = pd.DataFrame(columns = ['id_usuario',
                'fecha',
                'hora',
                'edad',
                'diastolica',
                'sistolica',
                'frecuencia_c',
                'temperatura',
                'saturacion',
                'sudoracion',
                'temblores',
                'diabetico',
                'glucosa'])

ahora = datetime.now()

lista =['000000000',str(ahora.strftime('%d/%m/%Y')),ahora.strftime('%H:%M'),0,0,0,0,0.00,0,0,0,0,0] 
df_historia.loc[0] = lista

df_historia.head()


,id_usuario,fecha,hora,edad,diastolica,sistolica,frecuencia_c,temperatura,saturacion,sudoracion,temblores,diabetico,glucosa
0,000000000,02/05/2023,15:42,0,0,0,0,0.0,0,0,0,0,0


In [10]:
df_historia.head()

,id_usuario,fecha,hora,edad,diastolica,sistolica,frecuencia_c,temperatura,saturacion,sudoracion,temblores,diabetico,glucosa
0,000000000,27/04/2023,16:16,0,0,0,0,0.0,0,0,0,0,0
1,000000000,27/04/2023,16:16,0,0,0,0,0.0,0,0,0,0,0
2,000000000,27/04/2023,16:16,0,0,0,0,0.0,0,0,0,0,0


In [14]:
df_h = pd.read_json('https://storage.googleapis.com/bucket_glucosa_bot/historia.json',encoding='utf-8')

In [15]:
df_h

,id_usuario,fecha,hora,edad,diastolica,sistolica,frecuencia_c,temperatura,saturacion,sudoracion,temblores,diabetico,glucosa
0,0,26/04/2023,13:29,0,0,0,0,0.0,0,0,0,0,0.000000
1,6077785968923242,29/04/2023,05:42,35,80,120,80,95.0,90,0,0,0,84.820000
2,6077785968923242,29/04/2023,06:09,25,80,120,80,96.8,80,0,0,1,52.200001
3,6077785968923242,29/04/2023,06:28,40,60,90,60,95.0,90,0,0,1,38.340000
4,6077785968923242,29/04/2023,12:47,25,60,120,99,98.6,90,0,0,0,84.059998
5,6077785968923242,29/04/2023,12:59,1,500,10,2,96.8,99,0,0,1,348.190002
6,6077785968923242,29/04/2023,15:50,100,80,120,80,95.0,99,0,0,1,108.190002
7,6077785968923242,29/04/2023,22:36,36,60,180,80,95.0,99,0,0,0,79.940002


In [24]:
df_usuario=df_h.loc[df_h['id_usuario'] == 6077785968923242]
len(df_usuario)

0

In [23]:
df_usuario.loc[len(df_usuario.loc[df_h['id_usuario'] == 6077785968923242])-1]['glucosa']

108.1900024414

In [7]:
len(df_historia)

2

In [9]:
df_historia.loc[len(df_historia)]=lista

In [3]:
print(ahora.strftime('%H:%M'))

13:29


In [4]:
df_historia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_usuario    1 non-null      object 
 1   fecha         1 non-null      object 
 2   hora          1 non-null      object 
 3   edad          1 non-null      int64  
 4   diastolica    1 non-null      int64  
 5   sistolica     1 non-null      int64  
 6   frecuencia_c  1 non-null      int64  
 7   temperatura   1 non-null      float64
 8   saturacion    1 non-null      int64  
 9   sudoracion    1 non-null      int64  
 10  temblores     1 non-null      int64  
 11  diabetico     1 non-null      int64  
 12  glucosa       1 non-null      int64  
dtypes: float64(1), int64(9), object(3)
memory usage: 112.0+ bytes


In [5]:
df_historia.to_json('historia.json')

In [6]:
historia_str = df_historia.to_json()
historia = json.loads(historia_str)

In [9]:
type(historia)

dict

In [59]:
historia["id_usuario"]["1"]={"4564564564"}

In [10]:
historia

{'id_usuario': {'0': '000000000'},
 'fecha': {'0': '26/04/2023'},
 'hora': {'0': '13:29'},
 'edad': {'0': 0},
 'diastolica': {'0': 0},
 'sistolica': {'0': 0},
 'frecuencia_c': {'0': 0},
 'temperatura': {'0': 0.0},
 'saturacion': {'0': 0},
 'sudoracion': {'0': 0},
 'temblores': {'0': 0},
 'diabetico': {'0': 0},
 'glucosa': {'0': 0}}

In [40]:
historia_df = pd.DataFrame(historia)

In [41]:
historia_df.head()

,id_usuario,fecha,hora,edad,diastolica,sistolica,frecuencia_c,temperatura,saturacion,sudoracion,temblores,diabetico,glucosa
0,000000000,25/04/2023,13:09,0,0,0,0,0.0,0,0,0,0,0


In [42]:
historia_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_usuario    1 non-null      object 
 1   fecha         1 non-null      object 
 2   hora          1 non-null      object 
 3   edad          1 non-null      int64  
 4   diastolica    1 non-null      int64  
 5   sistolica     1 non-null      int64  
 6   frecuencia_c  1 non-null      int64  
 7   temperatura   1 non-null      float64
 8   saturacion    1 non-null      int64  
 9   sudoracion    1 non-null      int64  
 10  temblores     1 non-null      int64  
 11  diabetico     1 non-null      int64  
 12  glucosa       1 non-null      int64  
dtypes: float64(1), int64(9), object(3)
memory usage: 112.0+ bytes


In [6]:
import pickle
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [11]:
#Carga de Dataset desde bucket GCP
df = pd.read_csv('https://storage.googleapis.com/dataset_bucket_bot/glucosa.csv',delimiter=';',encoding='utf-8')
df['diabetic'] = df['diabetic'].replace('D', 1).replace('N', 0)
X = df.drop('glucose', axis=1)
y = df['glucose']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', alpha=0.001, batch_size=32, verbose=False, tol=0.001)
mlp.fit(X_train, y_train)
y_preds = mlp.predict(X_test)
confusion_matrix = confusion_matrix(y_test, y_preds)
accuracy_score = accuracy_score(y_test, y_preds)
print(f'Percentaje de acierto: {accuracy_score*100}%')

TypeError: 'numpy.ndarray' object is not callable

In [4]:
import os, binascii
print(binascii.hexlify(os.urandom(25)))

b'c34e4e24ff2577d3e79d67fe07fa2567e56db48a1cfc020c77'


In [14]:
policy_text = open('Policy.txt',encoding="utf-8",newline=None).read()

In [16]:
print(policy_text)

Tratamiento de datos a través de nuestra FanPage de Facebook

Responsabilidad 

Operamos lo que se denomina una «FanPage» en la plataforma de red social Facebook, un servicio proporcionado por Facebook Ireland Ltd., 4 Grand Canal Square Grand Canal Harbour, Dublín 2, Irlanda, para comunicarnos con los visitantes activos en ella (= fans) que han marcado nuestra FanPage con un «Me gusta» y, entre otras cosas, informarles sobre nuestros productos y servicios. 
Hemos celebrado un acuerdo con Facebook sobre responsabilidad conjunta en virtud del art. 26 del RGPD, que regula la distribución de las obligaciones de protección de datos (https://www.facebook.com/legal/terms/page_controller_addendum). 
En particular, Facebook asume la obligación de informar a los visitantes, garantizar la seguridad de los datos y notificar violaciones de datos y es el contacto prioritario para que los propietarios de los datos ejerzan sus derechos. 
Datos de contacto de los corresponsables del tratamiento 

• Fac